## Loading the MobileNet model

Here, we will be freezing all the layers of the model, except the top 4 layers which we are gonna train with the monkey breed dataset

In [1]:
from keras.applications import MobileNet

# MobileNet was designed to work with 224x224 pixel images
rows, cols = 224, 224

model = MobileNet(weights = 'imagenet', 
                  include_top = False, 
                  input_shape = (rows, cols, 3))

# Now, we will freeze the layers
for layer in model.layers:
    layer.trainable = False

# Let's take a look at the layers
for (i, layer) in enumerate(model.layers):
    print(str(i), ". ", layer.__class__.__name__)

17227776/17225924 [==============================] - 96s 6us/step
0 .  InputLayer
1 .  ZeroPadding2D
2 .  Conv2D
3 .  BatchNormalization
4 .  ReLU
5 .  DepthwiseConv2D
6 .  BatchNormalization
7 .  ReLU
8 .  Conv2D
9 .  BatchNormalization
10 .  ReLU
11 .  ZeroPadding2D
12 .  DepthwiseConv2D
13 .  BatchNormalization
14 .  ReLU
15 .  Conv2D
16 .  BatchNormalization
17 .  ReLU
18 .  DepthwiseConv2D
19 .  BatchNormalization
20 .  ReLU
21 .  Conv2D
22 .  BatchNormalization
23 .  ReLU
24 .  ZeroPadding2D
25 .  DepthwiseConv2D
26 .  BatchNormalization
27 .  ReLU
28 .  Conv2D
29 .  BatchNormalization
30 .  ReLU
31 .  DepthwiseConv2D
32 .  BatchNormalization
33 .  ReLU
34 .  Conv2D
35 .  BatchNormalization
36 .  ReLU
37 .  ZeroPadding2D
38 .  DepthwiseConv2D
39 .  BatchNormalization
40 .  ReLU
41 .  Conv2D
42 .  BatchNormalization
43 .  ReLU
44 .  DepthwiseConv2D
45 .  BatchNormalization
46 .  ReLU
47 .  Conv2D
48 .  BatchNormalization
49 .  ReLU
50 .  DepthwiseConv2D
51 .  BatchNormalization
52

A total of 86 layers!

## Creating a function returning our FC Head

In [6]:
def addFCHead(rest_model, num_classes):
    """
        Creates the FC Head layer for the model which can
        be placed onto the bottom layers
    """
    top_model = rest_model.output
    top_model = (GlobalAveragePooling2D())(top_model)
    top_model = (Dense(1024, activation = 'relu'))(top_model)
    top_model = (Dense(256, activation = 'relu'))(top_model)
    top_model = (Dense(64, activation = 'relu'))(top_model)
    top_model = (Dense(num_classes, activation = 'sigmoid'))(top_model)
    
    return top_model

## Let's add our FC layers on top of MobileNet

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 10

FC_Head = addFCHead(model, num_classes)

model = Model(inputs = model.input, outputs = FC_Head)

print(model.summary())

ValueError: Input 0 of layer global_average_pooling2d_1 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 10]